# Linear Optimization Example

## Problem description

An aluminium’s alloy maker uses bauxite for producing 10 different alloy types. The production is divided into four sections: bauxite extraction, aluminium oxide transformation, electrolysis and sheet production. The characteristics of the production system are summarized in the following table: 

  Alloy      | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | hours available
  ---------------------|-----|---|----|---|----|------|---|----|----|---|---
  Extraction  | 0,02 | 0,04 | 0,02 | 0,02 | 0,02 | 0,02 | 0,01 | 0,02 | 0,01 | 0,01 | 600
  Transformation | 0,04 | 0,03 | 0,04 | 0,04 | 0,04 | 0,04 | 0,04 | 0,04 | 0,04 | 0,04 | 1200
  Electrolysis | 0,03 | 0,03 | 0,03 | 0,03 | 0,03 | 0,03 | 0,02 | 0,03 | 0,02 | 0,02 | 850
  Sheet production | 0,05 | 0,04 | 0,05 | 0,05 | 0,05 | 0,05 | 0,04 | 0,05 | 0,04 | 0,04 | 1500
  
  
In addition, 130000 kg bauxite field is available for production. It must be taken into account that a 1kg sheet of each aluminium alloy requires: 6, 5, 3, 8, 9, 4, 4, 2, 8 and 4 kg of bauxite, respectively. 

The maximum production in kg, minimum demand in kg and the individual cost and revenue per kg of alloy are specified in the following table: 

  Alloy      | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10
  -----------|-----|---|----|---|----|------|---|----|----|---
  Revenue (money/kg) | 100 | 120 | 75 | 110 | 90 | 60 | 180 | 85 | 110 | 120
  Cost (money/kg) | 30 | 35 | 25 | 32 | 30 | 20 | 50 | 25 | 31 | 34
  Maximum production (kg)| 4200 | 2500 | 8100 | 2200 | 6700 | 3600 | 3000 | 2500 | 2900 | 3800
  Demand (kg) | 3000 | 1000 | 5000 | 1500 | 2000 | 2500 | 1000 | 1200 | 1400 | 2000
  
 The goal of the maker is to optimize the benefits.

## Solution

### 1. Formulate the problem.

#### Summary
The goal of this problem is to find the optimal aluminium production for a company in order to maximize the benefits, given the costs associated with the production of each aluminium alloy, the minimum required quantity and the production capacity they have. 


#### Sets:

 Alloys = set of aluminium alloys  
 Steps = set of steps involved in the aluminium production (shared among all of them)
 
#### Parameters:

+ ci: Revenue - Cost obtained per kg of a given alloy i.
+ mi: kilograms of bauxite needed to produce a kilogram of alloy i.
+ m: total quantity (in kilograms) available in the field.
+ tci,j: time required (in hours) in the section j for the alloy i.
+ tj: total available time (in hours )in the section j.
+ Pi: maximum capacity (in kilograms) for a given alloy i. 
+ Di: minimum demand (in kilograms) for a given alloy i.

#### Variables: 

+ xi: quantity (in kilograms) produced of a given alloy i.

#### Objective function: 
The objective is to maximize the benefits of the company. Those are given adding the profits obtained with the alloy sells. 

\begin{align*}
\underset{x_{i}}{\max} & \quad \sum_{i=1}^{n}c_{i}x_{i}\
\end{align*}

#### Constraints:
\begin{align*}
  &\sum_{i=1}^{10} m_{i} * x_{i} \leq m \quad \\
  &\sum_{i=1}^{10} tc_{i,j} * x_{i} \leq t_{j} \quad \forall j\\
  &x_{i} \leq P{i} \quad \forall i\\
  &x_{i} \geq D{i} \quad \forall i\\
  &x_{i} \geq 0 \quad \forall i 
\end{align*}


There is only one constraint with a lower inequality, so we will change its sign.

\begin{align*}
  &\sum_{i=1}^{10} m_{i} * x_{i} \leq m \quad \\
  &\sum_{i=1}^{10} tc_{i,j} * x_{i} \leq t_{j} \quad \forall j\\
  &x_{i} \leq P{i} \quad \forall i\\
  &(-1) * x_{i} \leq (-1) * D{i} \quad \forall i\\
  &x_{i} \geq 0 \quad \forall i 
\end{align*}

### 2. Implement the model in Pyomo. Compute the sensitivities.

#### Pyomo implementation

In [5]:
%%writefile max_benefit_vehicles.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.n = Param(within=NonNegativeIntegers)
model.m = Param(within=NonNegativeIntegers)

model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)

model.vc = Param(model.I) #variable cost per kg and alloy
model.p = Param(model.I) #price per kg and alloy
model.hc = Param(model.I, model.J) #time cost per alloy and section
model.h = Param(model.J) #time limit per section
model.mc = Param(model.I) #bauxite cost per kg and alloy
model.tm = Param(within=NonNegativeReals) #total kg of bauxite available
model.u = Param(model.I) #maximum production per alloy
model.d = Param(model.I) #minimum demand per alloy

model.x = Var(model.I, domain=NonNegativeReals)

#definition of the objective function
def obj_expression(model): 
	return sum(model.x[i]*(model.p[i]-model.vc[i]) for i in model.I)

model.OBJ = Objective(rule=obj_expression, sense = maximize)

#maximum hour constraint
def hour_constraint_rule(model, j): 
     return sum(model.x[i]*model.hc[i,j] for i in model.I) <= model.h[j]

model.hour_Constraint = Constraint(model.J, rule=hour_constraint_rule)

#maximum material constraint
def material_constraint_rule(model): 
     return sum(model.x[i]*model.mc[i] for i in model.I) <= model.tm

model.material_Constraint = Constraint(rule=material_constraint_rule)

#maximum production constraint
def production_constraint_rule(model, i): 
     return model.x[i] <= model.u[i]

model.production_Constraint = Constraint(model.I, rule=production_constraint_rule)

#minimum demand constraint
def demand_constraint_rule(model, i): 
     return (-1)*model.x[i] <= model.d[i]

model.demand_Constraint = Constraint(model.I, rule=demand_constraint_rule)

Overwriting max_benefit_vehicles.py


After writing the code for the model, we create a .dat file with the data we are going to use.

In [61]:
%%writefile max_benefit_vehicles.dat

param n := 10; #number of alloys
param m := 4; #number of sections
    
param vc := 
1  30 
2  35 
3  25 
4  32 
5  30 
6  20 
7  50 
8  25 
9  34 
10 31
;

param p := 
1  100 
2  120 
3  75 
4  110 
5  90 
6  60 
7  180 
8  85 
9  120 
10 110
;

param hc : 1 2 3 4 :=
1  0.02 0.04 0.03 0.05 
2  0.04 0.03 0.03 0.04 
3  0.02 0.04 0.03 0.05 
4  0.02 0.04 0.03 0.05 
5  0.02 0.04 0.03 0.05 
6  0.02 0.04 0.03 0.05 
7  0.01 0.04 0.02 0.04 
8  0.02 0.04 0.03 0.05 
9  0.01 0.04 0.02 0.04 
10 0.01 0.04 0.02 0.04
;

param h :=
1 600 
2 1200 
3 850 
4 1500
;

param mc :=
1  6 
2  5 
3  3 
4  8 
5  9 
6  4 
7  4 
8  2 
9  8 
10 4
;

param tm := 130000;

param u :=
1  4200 
2  2500 
3  8100 
4  2200 
5  6700 
6  3600 
7  3000 
8  2500 
9  2900 
10 3800
;

param d :=
1  -3000 
2  -1000 
3  -5000 
4  -1500 
5  -2000 
6  -2500 
7  -1000 
8  -1200 
9  -1400 
10 -2000
;

Overwriting max_benefit_vehicles.dat


We will solve the problem in the shell (!pyomo) and print here the results (!type results.yml).

In [62]:
!pyomo solve  max_benefit_vehicles.py  max_benefit_vehicles.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.05] Applying solver
[    0.09] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 1988433.333333333
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=10, Index=I
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 :           3000.0 :  None : False : False : NonNegativeReals
          2 :     0 :           2500.0 :  None : False : False : NonNegativeReals
          3 :     0 : 5366.66666666666 :  None : False : False : NonNegativeReals
          4 :     0 :           1500.0 :  None : False : False : NonNegativeReals
          5 :     0 :           2000.0 :  None : False : False : NonNegativeReals
          6 :     0 :           2500.0 :  None : False : False : NonNegativeReals
          7 :     0 :     

In [63]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1988433.33333333
  Upper bound: 1988433.33333333
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 11
  Number of nonzeros: 71
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.031246662139892578
# ----------------------------------------------------------
#   Solution Information
# -----------

#### Sensitivities

We look mostly at the constraints that are bounded (those where we are at the boundaries of the search space, that is, the active constraints: material_Constraint, production_Constraint 2, 7, 8 and 10, demand_Constraint 1, 4, 5, 6 and 9). We will change in 1 unit each of the values b vector of the mentioned constraints and check how much the increment in the objective function is. That allows us to compute the sensitivities for these constraints. The highest sensibilitites for small increments are expected to appear at those constraints, but we check also the others. 
\begin{align*}
\lambda_{j} = \frac{\Delta z_{j}}{\Delta b_{j}}\
\end{align*}

In our case, the increment in b is always 1 so lambda equals z.

We have handmadely changed the .dat file for computing each of the sensitivities. Nevertheless, when the result file is checked, it will be observed that the dual problem solutions are the same that the obtained values in sensitivity computation. Hence, it is demonstrated that the Strong Duality Theorem is verified in our problem.

+ Time 1 --> 0

+ Time 2 --> 0

+ Time 3 --> 0

+ Time 4 --> 0

+ Material --> 16.67

+ Production 1 --> 0

+ Production 2 --> 1.67

+ Production 3 --> 0

+ Production 4 --> 0

+ Production 5 --> 0

+ Production 6 --> 0

+ Production 7 --> 63.33

+ Production 8 --> 26.67

+ Production 9 --> 0

+ Production 10 --> 12.33

+ Demand 1 --> 30

+ Demand 2 --> 0

+ Demand 3 --> 0

+ Demand 4 --> 55.33

+ Demand 5 --> 90

+ Demand 6 --> 26.67

+ Demand 7 --> 0

+ Demand 8 --> 0

+ Demand 9 --> 47.33

+ Demand 10 --> 0

Furthermore, the sensitivities computations provide us with important insights about the business. These results give evidences about in which part of the company I should invest my money if had the possibility of change any of the limits understood as constraints. 

For example, if increasing the capacity costs the same in the warehouse of any kind of alloys, the most profitable measure as director of the company is to invest in the alloy 7. 

On the other hand, if no cost had taken place when the minimum demand is not satisfied, it is better reduce the production of alloy 5, and use this material in alloy 3, probably. 

Finally, it shows us that in the current situation, buy extract 1 kilogram of bauxite is traduces if 16.67 monetary units when it is destinated to the correspond alloy.

#### **Problem agglutinated reformulation**

All the constraints (except the non-negative one) can be agglutinated in one matrix inequality, and the problems looks like:

\begin{align*}
\underset{x_{i}}{\max} & \quad \sum_{i=1}^{n}c_{i}x_{i}\
\end{align*}

\begin{align*}
  &\sum_{i=1}^{n} A_{j,i}* x_{i} \leq b_{j} \quad \forall j\\
  &x_{i} \geq 0 \quad \forall i 
\end{align*}

or

\begin{align*}
\underset{x}{\max} & \quad c^T x\
\end{align*}

\begin{align*}
  & A x \leq b \quad \\
  &x \geq 0 \quad \ 
\end{align*}

As we can see, when we code this agglutinated version of the equations, the result is the same (obviously). Notice that the c vector contains as entries the difference between revenues and costs for each single alloy

In [70]:
%%writefile max_benefit_vehicles.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.n = Param(within=NonNegativeIntegers)
model.m = Param(within=NonNegativeIntegers)

model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)

model.c = Param(model.I) #benefit per unit and model
model.A = Param(model.J, model.I) #hour cost per model and section
model.b = Param(model.J) #hour limit per section

model.x = Var(model.I, domain=NonNegativeReals)

#definition of the objective function
def obj_expression(model): 
	return sum(model.x[i]*model.c[i] for i in model.I)

model.OBJ = Objective(rule=obj_expression, sense = maximize)

#all constraint
def all_constraint_rule(model, j): 
     return sum(model.x[i]*model.A[j,i] for i in model.I) <= model.b[j]

model.all_Constraint = Constraint(model.J, rule=all_constraint_rule)

Overwriting max_benefit_vehicles.py


In [71]:
%%writefile max_benefit_vehicles.dat

param n := 10; #number of variables
param m := 25; #number of constraints
    
param c :=
1  70
2  85
3  50
4  78
5  60
6  40
7  130
8  60
9  86
10 79
;

param A : 1 2 3 4 5 6 7 8 9 10 :=
1  0.02 0.04 0.02 0.02 0.02 0.02 0.01 0.02 0.01 0.01 
2  0.04 0.03 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 
3  0.03 0.03 0.03 0.03 0.03 0.03 0.02 0.03 0.02 0.02 
4  0.05 0.04 0.05 0.05 0.05 0.05 0.04 0.05 0.04 0.04 
5     6    5    3    8    9    4    4    2    8    4 
6     1    0    0    0    0    0    0    0    0    0 
7     0    1    0    0    0    0    0    0    0    0 
8     0    0    1    0    0    0    0    0    0    0 
9     0    0    0    1    0    0    0    0    0    0 
10    0    0    0    0    1    0    0    0    0    0 
11    0    0    0    0    0    1    0    0    0    0 
12    0    0    0    0    0    0    1    0    0    0 
13    0    0    0    0    0    0    0    1    0    0 
14    0    0    0    0    0    0    0    0    1    0 
15    0    0    0    0    0    0    0    0    0    1 
16   -1    0    0    0    0    0    0    0    0    0
17    0   -1    0    0    0    0    0    0    0    0
18    0    0   -1    0    0    0    0    0    0    0
19    0    0    0   -1    0    0    0    0    0    0
20    0    0    0    0   -1    0    0    0    0    0
21    0    0    0    0    0   -1    0    0    0    0
22    0    0    0    0    0    0   -1    0    0    0
23    0    0    0    0    0    0    0   -1    0    0
24    0    0    0    0    0    0    0    0   -1    0
25    0    0    0    0    0    0    0    0    0   -1
;

param b :=
1  600
2  1200
3  850
4  1500
5  130000
6  4200
7  2500
8  8100
9  2200
10 6700
11 3600
12 3000
13 2500
14 2900 
15 3800
16 -3000
17 -1000 
18 -5000
19 -1500
20 -2000
21 -2500 
22 -1000
23 -1200
24 -1400
25 -2000
;

Overwriting max_benefit_vehicles.dat


In [72]:
!pyomo solve  max_benefit_vehicles.py  max_benefit_vehicles.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.01] Applying solver
[    0.10] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 1988433.333333333
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=10, Index=I
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 :           3000.0 :  None : False : False : NonNegativeReals
          2 :     0 :           2500.0 :  None : False : False : NonNegativeReals
          3 :     0 : 5366.66666666666 :  None : False : False : NonNegativeReals
          4 :     0 :           1500.0 :  None : False : False : NonNegativeReals
          5 :     0 :           2000.0 :  None : False : False : NonNegativeReals
          6 :     0 :           2500.0 :  None : False : False : NonNegativeReals
          7 :     0 :     

### 3. Formulate the dual problem.

#### Formulation

To formulate the dual problem we simply follow the rules shown in the lessons. In the compact form, the dual problem looks like:

##### Objective function: 
\begin{align*}
\underset{y}{\min} & \quad b^T y\
\end{align*}

##### Constraints:
\begin{align*}
  & A^T y \geq c \quad \\
  &y \geq 0 \quad \
\end{align*}


Where the matrices and vectors A, b and c are the same as the reformulated ones in part b).

To code the dual problem in pyomo we use the structure developed in b) but with the proper changes. 

In [76]:
%%writefile max_benefit_vehicles_dual.py

from __future__ import division 
from pyomo.environ import *

dualmodel = AbstractModel()

dualmodel.n = Param(within=NonNegativeIntegers)
dualmodel.m = Param(within=NonNegativeIntegers)

dualmodel.I = RangeSet(1, dualmodel.n)
dualmodel.J = RangeSet(1, dualmodel.m)

dualmodel.A =Param(dualmodel.J, dualmodel.I)
dualmodel.b = Param(dualmodel.J)
dualmodel.c = Param(dualmodel.I)

dualmodel.y = Var(dualmodel.J, domain=NonNegativeReals)

#definition of the objective function
def obj_expression(dualmodel): 
	return sum(dualmodel.y[j]*dualmodel.b[j] for j in dualmodel.J)

dualmodel.OBJ = Objective(rule=obj_expression, sense = minimize)

#Constraints
def all_constraint_rule(dualmodel, i): 
     return sum(dualmodel.A[j,i]*dualmodel.y[j] for j in dualmodel.J) >= dualmodel.c[i]
    
dualmodel.all_Constraint = Constraint(dualmodel.I, rule=all_constraint_rule)

Overwriting max_benefit_vehicles_dual.py


The .dat file with the data is exactly the same as in the primal problem, but we will write it here again!

In [77]:
%%writefile max_benefit_vehicles_dual.dat

param n := 10; #number of variables
param m := 25; #number of constraints
    
param c :=
1  70
2  85
3  50
4  78
5  60
6  40
7  130
8  60
9  86
10 79
;

param A : 1 2 3 4 5 6 7 8 9 10 :=
1  0.02 0.04 0.02 0.02 0.02 0.02 0.01 0.02 0.01 0.01 
2  0.04 0.03 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 
3  0.03 0.03 0.03 0.03 0.03 0.03 0.02 0.03 0.02 0.02 
4  0.05 0.04 0.05 0.05 0.05 0.05 0.04 0.05 0.04 0.04 
5     6    5    3    8    9    4    4    2    8    4 
6     1    0    0    0    0    0    0    0    0    0 
7     0    1    0    0    0    0    0    0    0    0 
8     0    0    1    0    0    0    0    0    0    0 
9     0    0    0    1    0    0    0    0    0    0 
10    0    0    0    0    1    0    0    0    0    0 
11    0    0    0    0    0    1    0    0    0    0 
12    0    0    0    0    0    0    1    0    0    0 
13    0    0    0    0    0    0    0    1    0    0 
14    0    0    0    0    0    0    0    0    1    0 
15    0    0    0    0    0    0    0    0    0    1 
16   -1    0    0    0    0    0    0    0    0    0
17    0   -1    0    0    0    0    0    0    0    0
18    0    0   -1    0    0    0    0    0    0    0
19    0    0    0   -1    0    0    0    0    0    0
20    0    0    0    0   -1    0    0    0    0    0
21    0    0    0    0    0   -1    0    0    0    0
22    0    0    0    0    0    0   -1    0    0    0
23    0    0    0    0    0    0    0   -1    0    0
24    0    0    0    0    0    0    0    0   -1    0
25    0    0    0    0    0    0    0    0    0   -1
;

param b :=
1  600
2  1200
3  850
4  1500
5  130000
6  4200
7  2500
8  8100
9  2200
10 6700
11 3600
12 3000
13 2500
14 2900 
15 3800
16 -3000
17 -1000 
18 -5000
19 -1500
20 -2000
21 -2500 
22 -1000
23 -1200
24 -1400
25 -2000
;

Overwriting max_benefit_vehicles_dual.dat


In [78]:
!pyomo solve  max_benefit_vehicles_dual.py  max_benefit_vehicles_dual.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.03] Applying solver
[    0.11] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 1988433.333333338
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    y : Size=25, Index=J
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 :              0.0 :  None : False : False : NonNegativeReals
          2 :     0 :              0.0 :  None : False : False : NonNegativeReals
          3 :     0 :              0.0 :  None : False : False : NonNegativeReals
          4 :     0 :              0.0 :  None : False : False : NonNegativeReals
          5 :     0 : 16.6666666666667 :  None : False : False : NonNegativeReals
          6 :     0 :              0.0 :  None : False : False : NonNegativeReals
          7 :     0 : 1.66

In [79]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1988433.33333333
  Upper bound: 1988433.33333333
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 26
  Number of nonzeros: 71
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.031253814697265625
# ----------------------------------------------------------
#   Solution Information
# -----------

Finally, by analyzing the dual problem results, we obtain again the sensibilities and the same optimum value for the objective function computed in the previous section, what verifies the Strong Duality Theorem.

### 4. Impose a conditional constraints (linear) that require the use of binary or integer variables.

Imagine that the company is introducing in the market their latest aluminium alloy: the number 9. This alloy is supposed to replace the number 10, as it is the logical evolution: similar specifications, but with a few improvements and a slighty higher price that translates in a higher revenue for the company. However, the new model adoption remains a bit low according to the recent results, and consequently they want to introduce some conditions to their production plan in order to keep the model 9 production at a higher rate over the model 10, even if that means a likely lower revenue (the company predicts that a higher interest in the model 10, independently of what happens with the model 9, is going to translate into a higher demand of model 9 in the mid term). 

The currently expected minimum demand for the model 9 is 1400, and the maximum production capacity 20000. These numbers are quite similar to the ones of the model 10: minimum of 2000 and a maximum of 20000. The company wants to address the following policy: if the production plan that gives the highest revenue implies that more kilograms of aluminium of the alloy 10 than the alloy 9 are produced, then the production of the model 10 should be capped at the demand level of the model 9. This might translate in a severe decrease of the production of the model 10 in case the optimum solution implied a very high model 10 production; if this was not the case, or the model 10 production was lower than the model 9, this logical constraint will not affect the production plan.

To code this logical constraint in a mathematical language we can use the following strategy based on a binary variable y:

If:
\begin{align*}
x_{9} - x_{10} \leq 0 \qquad \Leftrightarrow \qquad x_{10} - x_{9} \geq 0
\end{align*}
Then:
\begin{align*}
x_{9} \leq D_{10}
\end{align*}

Using a binary variable:

If:
\begin{align*}
x_{10} - x_{9} -My \leq 0
\end{align*}
Then:
\begin{align*}
x_{9} -M(1-y) \leq D_{10}
\end{align*}
Where:
\begin{align*}
y=\left\{\begin{matrix}
1 \quad \text{if constraint is satisfied}\\
0 \quad \text{if constraint is not satisfied} 
\end{matrix}\right.
\end{align*}

Then, to solve the new problem we only need to add this rules to the constraints set and recompute the solution. 

In [19]:
%%writefile max_benefit_vehicles_with_logical_constraint.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

model.n = Param(within=NonNegativeIntegers)
model.m = Param(within=NonNegativeIntegers)

model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.m)

model.vc = Param(model.I) #variable cost per unit and model
model.p = Param(model.I) #price per unit and model
model.hc = Param(model.I, model.J) #hour cost per model and section
model.h = Param(model.J) #hour limit per section
model.mc = Param(model.I) #material cost per unit and model
model.tm = Param(within=NonNegativeIntegers) #total material available
model.u = Param(model.I) #maximum production per model
model.d = Param(model.I) #minimum demand per model

model.x = Var(model.I, domain=NonNegativeIntegers)
# NEW VARIABLE: binary variably y to model the logical constraint
model.y = Var(domain=Binary)

#definition of the objective function
def obj_expression(model): 
	return sum(model.x[i]*(model.p[i]-model.vc[i]) for i in model.I)

model.OBJ = Objective(rule=obj_expression, sense = maximize)

#maximum hour constraint
def hour_constraint_rule(model, j): 
     return sum(model.x[i]*model.hc[i,j] for i in model.I) <= model.h[j]

model.hour_Constraint = Constraint(model.J, rule=hour_constraint_rule)

#maximum material constraint
def material_constraint_rule(model): 
     return sum(model.x[i]*model.mc[i] for i in model.I) <= model.tm

model.material_Constraint = Constraint(rule=material_constraint_rule)

#maximum production constraint
def production_constraint_rule(model, i): 
     return model.x[i] <= model.u[i]

model.production_Constraint = Constraint(model.I, rule=production_constraint_rule)

#minimum demand constraint
def demand_constraint_rule(model, i): 
     return (-1)*model.x[i] <= model.d[i]

model.demand_Constraint = Constraint(model.I, rule=demand_constraint_rule)

# NEW RULE: logical constraint 
# If this:
def logical_constraint_rule_if(model): 
     return model.x[10] - model.x[9] -100000*model.y <= 0

model.logical_Constraint_if = Constraint(rule=logical_constraint_rule_if)

# Then this:
def logical_constraint_rule_then(model): 
     return model.x[9] -100000*(1-model.y) <= model.d[10]

model.logical_Constraint_then = Constraint(rule=logical_constraint_rule_then)

Overwriting max_benefit_vehicles_with_logical_constraint.py


We rewrite the data to solve the problem. As the reader can see, we have used the first formulation (the one that is less compact) in order to make the addition of new constraints more clear. Therefore, we use the according data formulation.

In [20]:
%%writefile max_benefit_vehicles_with_logical_constraint.dat

param n := 10; #number of models
param m := 4; #number of sections
    
param vc := 
1  30 
2  35 
3  25 
4  32 
5  30 
6  20 
7  50 
8  25 
9  34 
10 31
;

param p := 
1  100 
2  120 
3  75 
4  110 
5  90 
6  60 
7  180 
8  85 
9  120 
10 110
;

param hc : 1 2 3 4 :=
1  0.02 0.04 0.03 0.05 
2  0.04 0.03 0.03 0.04 
3  0.02 0.04 0.03 0.05 
4  0.02 0.04 0.03 0.05 
5  0.02 0.04 0.03 0.05 
6  0.02 0.04 0.03 0.05 
7  0.01 0.04 0.02 0.04 
8  0.02 0.04 0.03 0.05 
9  0.01 0.04 0.02 0.04 
10 0.01 0.04 0.02 0.04
;

param h :=
1 600 
2 1200 
3 850 
4 1500
;

param mc :=
1  6 
2  5 
3  3 
4  8 
5  9 
6  4 
7  4 
8  2 
9  8 
10 4
;

param tm := 130000;

param u :=
1  4200 
2  2500 
3  8100 
4  2200 
5  6700 
6  3600 
7  3000 
8  2500 
9  2900 
10 3800
;

param d :=
1  -3000 
2  -1000 
3  -5000 
4  -1500 
5  -2000 
6  -2500 
7  -1000 
8  -1200 
9  -1400 
10 -2000
;

Overwriting max_benefit_vehicles_with_logical_constraint.dat


In [21]:
!pyomo solve  max_benefit_vehicles_with_logical_constraint.py  max_benefit_vehicles_with_logical_constraint.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.07] Applying solver
[    0.24] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 1937830.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=10, Index=I
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          1 :     0 : 3000.0 :  None : False : False : NonNegativeIntegers
          2 :     0 : 2498.0 :  None : False : False : NonNegativeIntegers
          3 :     0 : 6170.0 :  None : False : False : NonNegativeIntegers
          4 :     0 : 1500.0 :  None : False : False : NonNegativeIntegers
          5 :     0 : 2000.0 :  None : False : False : NonNegativeIntegers
          6 :     0 : 2500.0 :  None : False : False : NonNegativeIntegers
          7 :     0 : 3000.0 :  None : False : False : NonNegativeIntegers
          8 

Prior to applying this logical constraint, the optimum solution implied a higher production of alloy 10 (due to its revenue beign more interesting). Now, with the new model, we can see how the production of model 10 is capped at the minimum level of model 9, forcing this one to increase its production up to 2000 kilograms, and therefore reaching the equilibrium between both alloy. However, and given that model 9 is not so interesting revenue-wise at the moment, the increase in model 9 only reaches the demand quantity and does not use all the resources that the decrease in alloy 10 left unused. Consequently, we can see that, in the new solution, apart from the changes in the production of 10 and 9, there is a huge increase in the production of number 3. 